In [383]:
import json
import numpy as np
import pandas as pd
import os

from math import cos, sin, exp, sqrt, pi, e

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

## Constants to be used across the notebook

In [384]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")    
with open(simulation_values_path, "r") as f:
    simulation_values = json.load(f)
    
simulation_values

{'DT_evap': 8.0,
 'Tdhw': 55.0,
 'Tmax_i': 60.0,
 'Tmin_i': 45.0,
 'Ts1': 23.0,
 'Ts2': 29.0,
 'cap_source1': 20,
 'cap_source2': 20,
 'depth_aquifer': 30.0,
 'n_buildings_MFH': 1600.0,
 'n_buildings_SFH': 0.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1,

In [385]:
load_or_write_new = int(input("Press 1 if you want to reload previous simulation values"))

if load_or_write_new == 1:    
    Tmin_i = simulation_values["Tmin_i"]
    Tmax_i = simulation_values["Tmax_i"]

    Tdhw = simulation_values["Tdhw"]

    Ts1 = simulation_values["Ts1"]
    Ts2 = simulation_values["Ts2"]

    DT_evap = simulation_values["DT_evap"]
    
    n_buildings_SFH = simulation_values["n_buildings_SFH"]
    n_buildings_MFH = simulation_values["n_buildings_MFH"]
    
    cap_source1= simulation_values['cap_source1']
    cap_source2= simulation_values['cap_source2']
    
else:
    # Min and Max temperatures to be received by the user.
    # Values [40,...,60]
    Tmin_i = float(input("Enter min setpoint temperature: "))
    Tmax_i = float(input("Enter max setpoint temperature: "))

    # Domestic hot water temperature.
    # Values [45,...,55]
    Tdhw = float(input("Enter domestic hot water temperature: "))

    # Our model supposes that the system has two different sources of heat.
    # Values [20,...,30]
    Ts1 = float(input("Temperature of source 1: "))
    Ts2 = float(input("Temperature of source 2: "))

    # difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
    # Values [1,...,10]
    DT_evap = float(input("Temperature difference among the supply and return pipes: "))
    
    # For the data visualization course, we will select 0 SFH & 1600 MFH buildings, which will cover an area of 1 km2
    n_buildings_SFH =float(input("Enter the number of SFH buildings: #"))
    n_buildings_MFH=float(input("Enter the number of MFH buildings:#"))
    
    cap_source1= int(input("Enter the thermal capacity of source 1 in MW:"))
    cap_source2= int(input("Enter the thermal capacity of source 2 in MW:"))

    
print(f"* {Tmin_i=}\n"
      f"* {Tmax_i=}\n"
      f"* {Tdhw=}\n"
      f"* {Ts1=}\n"
      f"* {Ts2=}\n"
      f"* {DT_evap=}\n"
      f"* {n_buildings_SFH=}\n"
      f"* {n_buildings_MFH=}\n"
      f"* {cap_source1=}\n"
      f"* {cap_source2=}\n"
     )

Press 1 if you want to reload previous simulation values 1


* Tmin_i=45.0
* Tmax_i=60.0
* Tdhw=55.0
* Ts1=23.0
* Ts2=29.0
* DT_evap=8.0
* n_buildings_SFH=0.0
* n_buildings_MFH=1600.0
* cap_source1=20
* cap_source2=20



# Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [386]:
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s1_schedule = np.array(simulation_values["s1_schedule"])
else:    
    s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

* s1_schedule_path='./data/private/s1_source_fake_schedule.xlsx'
* s1_schedule=array([[0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0]])
* s1_schedule[6,1]=0
* s1_schedule[7,1]=1



In [387]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s2_schedule = np.array(simulation_values["s2_schedule"])
else: 
    s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

* s2_schedule_path='./data/private/s2_source_fake_schedule.xlsx'
* s2_schedule=array([[1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0]])
* s2_schedule[15,1]=1
* s2_schedule[16,1]=0



## Take note of the values for the current simulation

In [388]:
simulation_values["Tmin_i"] = Tmin_i
simulation_values["Tmax_i"] = Tmax_i
simulation_values["Tdhw"] = Tdhw
simulation_values["Ts1"] = Ts1
simulation_values["Ts2"] = Ts2
simulation_values["DT_evap"] = DT_evap
simulation_values["s1_schedule"] = s1_schedule.tolist()
simulation_values["s2_schedule"] = s2_schedule.tolist()
simulation_values["n_buildings_SFH"] = n_buildings_SFH
simulation_values["n_buildings_MFH"] = n_buildings_MFH
simulation_values['cap_source1']  = cap_source1
simulation_values['cap_source2'] =  cap_source2

simulation_values

{'DT_evap': 8.0,
 'Tdhw': 55.0,
 'Tmax_i': 60.0,
 'Tmin_i': 45.0,
 'Ts1': 23.0,
 'Ts2': 29.0,
 'cap_source1': 20,
 'cap_source2': 20,
 'depth_aquifer': 30.0,
 'n_buildings_MFH': 1600.0,
 'n_buildings_SFH': 0.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1,

## Read the dataset

In [389]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

In [390]:
dataset_to_work = "fresno_california"
dataset_to_work

'fresno_california'

In [391]:
Tamb_hourly = AVAILABLE_DATASETS[dataset_to_work]

## Functional unit composition

The single-family houses (SFH) and multi-family houses (MFH) models have a fixed geometry for all the climates and periods of construction. It has been defined following the common characteristics for a European buildings. The SFH building model is composed of two storeys with a total of 100 m² of living area . The small-MFH correspond to a reference residential building that has two dwellings per floor, with a total floor area of 500 m2. (Inspire project, D2.1c(2014))

A functional unit of 1 km2 of land composed by a residential area of 1600 buildings of the type small-MFH was selected. This reference area has been already investigated in previous publications for the scenario assessment of low-temperature DH networks, as a typical high-density urban zone.


### Residential building typologies: SFH and MFH

#### Single-Family units

<img src="./images/SFH.png">        





#### Multi-Families units

<img src="./images/MFH.png">




### District heating system diagram

District heating (DH) is an energy service based on moving heat from available heat sources to immediate customers through a piping network. Nowadays, the fundamental idea of DH is to use LOCAL sources that would otherwise be WASTED in order to satisfy customers'needs (Werner, 2013). 


Traditional DH systems operate typically at temperatures far from the ambient temperature (80°C or higher) giving rise to high thermal losses and the need of costly piping insulation. Examples of traditional DH sources are cogeneration plants, industrial waste heat and incinerators, where heat is a by-product of other industrial processes (Cozzini, 2018).

New concepts in the DH sector propose lowering the operating temperatures further to a level equivalent to the ambient temperature, with an average of 20°C. Advantages of a system of this kind, are the reduction in thermal losses in the distribution system, the direct exploitation of available waste heat sources (refrigeration units in shopping malls, supermarkets, datacenters) and ground sources (aquifer wells), and the possibility of providing not only heating but also cooling through the use of reversible heat pumps on the buildings substations, as presented in the Figure.

<img src="./images/DHC_system.png">

### Space heating demand

The methodology used in this project is a bottom-up approach for the calculation of the total heat demand for space heating (SH) and domestic hot water (DHW) utilization of a district located in 4 different climates across Europe, using data from the Inspire project. The data includes annual energy consumption for each building typology in kWh/m2-y, SFHs and MFHs correspond to reference buildings in each climatic zone built between 1945-1970, non-refurbished. In this particular case, the climatic zones and reference cities considered are:

* Stuttgart, Germany for Continental climate
* London, United Kingdom for Oceanic climate
* Madrid, Spain for Southern-dry climate
* Rome, Italy for Mediterranean climate

This calculation is based on the number of SFH ($ |SFH| $) and MFH ($ |MFH| $) units. It also depends on the total heat needed by SFH ($ TOT_{SFH}$) and MFH ($ TOT_{MFH}$) units. The estimation of the total heated area for buildings within the functional unit ($ HD $) is:

\begin{align*}
HD = |SFH| * 100 * TOT_{SFH} + |MFH| * 500 * TOT_{MFH}
\end{align*}

It's important to stress that this calculation is done on a year basis.

In [392]:
Tamb_hourly["Heat_demand"] = (n_buildings_SFH * 100 * Tamb_hourly["SFH_bldg_tot"]) + (n_buildings_MFH * 500 * Tamb_hourly["MFH_bldg_tot"])
Tamb_hourly["SH_demand"] = Tamb_hourly["Heat_demand"] * Tamb_hourly["%SH_y"]
Tamb_hourly["DHW_demand"] = Tamb_hourly["Heat_demand"] * Tamb_hourly["%DHW_y"] # Equivalently, we can change Tamb_hourly["%DHW_y"] with (1 - Tamb_hourly["%SH_y"])

### Space heating distribution

In order to analyse the performance of this particular DH system, it is fundamental not only the spatial distribution of the heat demand, but also the distribution over time. For this purpose, the space heating hourly profile for each location is retrieved with a time dependency according to the heating degree method, known as the "Integration method".

This method accounts the amount (in degrees) and for how long (in hours) thermal heat is required to keep the indoor building temperature at a comfortable level, which will vary depending on different climates. The base temperature selected was set to 15 °C.

More info on calculation method: https://www.degreedays.net/introduction

In [393]:
def heating_degree(T_amb):
    T_base = 15
    return max(0, T_base - T_amb)

In [394]:
Tamb_hourly["heating_degree_days"] = Tamb_hourly.air_temp.apply(heating_degree)
Tamb_hourly["space_heating_dist"] = Tamb_hourly.heating_degree_days / max(sum(Tamb_hourly.heating_degree_days), 1)
Tamb_hourly["SH_consumption"] = Tamb_hourly["SH_demand"] * Tamb_hourly["space_heating_dist"]

### Domestic hot water consumption

Domestic hot water consumption is not weather influenced, and its variation is almost constant over the year. The hourly demand for hot water is estimated by the percentage of the total heat demand ($ HD_y $) is hot water $ DHW_y $. This data is also obtained from the INSPIRE database for different climatic zones. 

The total domestic hot water demand is evenly distributed over each day of the year, and then its hourly distribution is obtained by multiplying the daily needs by an hourly random profile. 

\begin{align*}
DHW_{d} = \frac{HD_{y}*DHW_{\%y}}{d}
\end{align*}

The DHW consumption is calculated in an hourly fashion.

In [395]:
Tamb_hourly["DHW_consumption"] = Tamb_hourly["DHW_demand"] * Tamb_hourly["DHW_hourly_consumption_ratio"] / 365

### Total heat demand 

Thus, the aggregated heat demand corresponds to the sum of the thermal heat requirements for SH and DHW, according to the equation:

\begin{align*}
HD_{TOT, h} = HD_{SH, h} + HD_{DHW, h}
\end{align*}



In [396]:
Tamb_hourly["Total_consumption"] = Tamb_hourly["DHW_consumption"] + Tamb_hourly["SH_consumption"]

In [397]:
Tamb_hourly["SH_consumption"].shape

(8759,)

## Temperature levels required for SH and DHW

The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [398]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [399]:
Tamb_hourly['space_heating_temp'] = Tamb_hourly.air_temp.apply(climatic_curve)
Tamb_hourly["hot_water_temp"] = Tdhw

## Calculate User Temperature

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [400]:
sh_consumption_ratio = Tamb_hourly["SH_consumption"] / Tamb_hourly["Total_consumption"]
dhw_consumption_ratio = Tamb_hourly["DHW_consumption"] / Tamb_hourly["Total_consumption"]

In [401]:
#Tamb_hourly["user_temp_2"] = Tamb_hourly.apply(Tuser, axis=1)
Tamb_hourly["user_temp"] = (sh_consumption_ratio * Tamb_hourly.space_heating_temp 
                            + dhw_consumption_ratio * Tamb_hourly.hot_water_temp)

## Calculate Sources Temperature based on Working Hours

In [402]:
dayofweeks = Tamb_hourly.index.dayofweek
hours = Tamb_hourly.index.hour

In [403]:
Tamb_hourly["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
Tamb_hourly["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]


## Calculate Network temperature

In [404]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.min([temp_s1, temp_s2])

In [405]:
Tamb_hourly["net_temp"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)

# Coefficient of performance (COP)

Heat pumps located at the users' substations are responsible for lifting the network temperature to a desired level for SH and DHW consumptions. The performance of the overall system was assessed according to the following coefficient of performance (COP) function:

\begin{align*}
COP\ = \eta_{m} COP_{C} +1 - \eta_{m}
\end{align*}

\begin{align*}
COP_{C}= \frac{T_{c}}{T{_c} - T_{e}} 
\end{align*}

\begin{align*}
T_{c}= T_{c,o} +  \Delta T_{HEX} 
\end{align*}

\begin{align*}
T_{e}=  T_{e,o} -  \Delta T_{HEX} 
\end{align*}

Where $COP_{C}$ corresponds to the Carnot COP which is a function of the condenser ($ T_{c}$) and evaporator ($T_{e}$) refrigerant temperatures respectively. These variables are estimated as the external fluid outlet temperatures, adjusted by a temperature drop ($\Delta T_{HEX}$) at the heat pump condenser and evaporator.  

In [406]:
DT_hx = 2.5
n_HP = 0.53
Te_o = Tamb_hourly.net_temp - DT_evap - DT_hx
Tc_o = Tamb_hourly.user_temp

Tamb_hourly['COP']= 1-n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)

# Heat losses

In [407]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 0.848  #Average heat loss of pre-insulated pipes along a network of 2km [MW/ K] 
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U  # Heat losses supply pipe [MW]
    HL_r = (T_ret - T_gr) * U  # Heat losses return pipe [MW]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [408]:
x = Tamb_hourly.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')

In [409]:
Tamb_hourly['E_loss_s'] = x['E_loss_s']
Tamb_hourly['E_loss_r'] = x['E_loss_r']
Tamb_hourly['E_loss_tot'] = x['E_loss_tot']

# Electricity consumption

The hourly electricity consumption by climatic zone is estimated as a function of the total thermal demand and the coefficient of performance of the systems substations:

\begin{align*}
E_{el} = \frac{E_{th}}{COP}
\end{align*}

In [410]:
Tamb_hourly['E_el']= Tamb_hourly['Total_consumption']/ Tamb_hourly['COP']
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,E_el,season,hour,dayofweek,source1_cap,source2_cap,heat_source1,heat_source2,heat_aquifer,date
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,5.5,1,1,2,8.182589,12.474162,17.903901,fresno_california,Southern dry,21.8,...,7191.9,winter,1,4,20000,20000,20000.0,6704.815088,0.0,2010-01-01
2010-01-01 02:00:00,5.3,1,1,3,8.180475,12.471033,17.903901,fresno_california,Southern dry,21.8,...,7429.83,winter,2,4,20000,20000,20000.0,7176.600294,0.0,2010-01-01
2010-01-01 03:00:00,5.1,1,1,4,8.178367,12.467906,17.903901,fresno_california,Southern dry,21.8,...,7670.78,winter,3,4,20000,20000,20000.0,7645.351317,0.0,2010-01-01
2010-01-01 04:00:00,4.9,1,1,5,8.176264,12.464783,17.903901,fresno_california,Southern dry,21.8,...,7914.75,winter,4,4,20000,20000,20000.0,8111.095117,0.0,2010-01-01
2010-01-01 05:00:00,4.8,1,1,6,8.174166,12.461662,17.903901,fresno_california,Southern dry,21.8,...,8037.85,winter,5,4,20000,20000,20000.0,8342.848752,0.0,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,8.1,365,12,8756,8.193230,12.492995,17.903901,fresno_california,Southern dry,21.8,...,5227.62,winter,19,4,20000,20000,20000.0,1823.051839,0.0,2010-12-31
2010-12-31 20:00:00,7.4,365,12,8757,8.191092,12.489849,17.903901,fresno_california,Southern dry,21.8,...,5655.17,winter,20,4,20000,20000,20000.0,3697.737657,0.0,2010-12-31
2010-12-31 21:00:00,6.9,365,12,8758,8.188958,12.486706,17.903901,fresno_california,Southern dry,21.8,...,5646.13,winter,21,4,20000,20000,20000.0,3420.985034,0.0,2010-12-31


# Thermal power provided by sources

District heating systems have an opportunity to use heat supply from several sources at the same time. In order for this to happen, it is necessary a control system that allows many thermal plants to deliver heat at the same distribution system. In this case, the base load can be supplied by up to two sources of waste heat, with a maximum capacity and temperature levels determined by the user of this tool. A third source consisting in aquifer wells provides heat from a depth of about 30 m when the base load plants are not operating, as well as in peak load conditions. 



In [411]:
Tamb_hourly["source1_cap"] = cap_source1 * 1000 * s1_schedule[hours, dayofweeks] # values in kW to match the thermal demand calculations
Tamb_hourly["source2_cap"] = cap_source2 * 1000 *  s2_schedule[hours, dayofweeks] # values in kW to match the thermal demand calculations


In [412]:
def calculate_heatsupply(source1_cap, source2_cap, source1_temp, source2_temp,thermal_energy_from_sources):
    if source1_cap == 0.0 and source2_cap == 0.0:
        heat_source1 = 0.0
        heat_source2= 0.0
        heat_aquifer = thermal_energy_from_sources
        return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])    
    elif source1_cap == 0.0:                    #Si la primera fuente no funciona, la otra cubre la demanda hasta su max capacidad
        if thermal_energy_from_sources <=  source2_cap:
            heat_source1 = 0.0
            heat_source2 = thermal_energy_from_sources
            heat_aquifer = 0.0
        else:
            heat_source1 = 0.0
            heat_source2 = source2_cap
            heat_aquifer = thermal_energy_from_sources - source2_cap
        return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])    
    elif source2_cap == 0.0:                                 #Si la segunda fuente no funciona, la otra cubre la demanda hasta su max capacidad
        if thermal_energy_from_sources <=  source1_cap:
            heat_source1 = thermal_energy_from_sources
            heat_source2 = 0.0
            heat_aquifer = 0.0
        else:
            heat_source1 = source1_cap
            heat_source2 = 0.0
            heat_aquifer = thermal_energy_from_sources - source1_cap
        return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])    
    else:                            #Si ambas plantas funcionan, opera la de mayor temperatura hasta su max capacidad, si no es suficiente, se activa la segunda planta
        if source1_temp < source2_temp:
            
            if thermal_energy_from_sources <=  source1_cap:
                heat_source1 = thermal_energy_from_sources
                heat_source2 = 0.0
                heat_aquifer = 0.0
            else:
                heat_source1 = source1_cap
                heat_source2 = thermal_energy_from_sources - source1_cap
                heat_aquifer = 0.0
            
            return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])           
        else: 
            if thermal_energy_from_sources <=  source2_cap:
                heat_source1 = 0.0
                heat_source2 = thermal_energy_from_sources
                heat_aquifer = 0.0
            else:
                heat_source1 = thermal_energy_from_sources - source2_cap
                heat_source2 = source2_cap
                heat_aquifer = 0.0
            return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])

In [413]:
y = Tamb_hourly.apply(lambda fila: calculate_heatsupply(fila.source1_cap,fila.source2_cap, fila.source1_temp, fila.source2_temp, fila.Total_consumption - fila.E_el), axis=1 , result_type='expand')
Tamb_hourly["heat_source1"] = y['heat_source1']
Tamb_hourly["heat_source2"] = y['heat_source2']
Tamb_hourly["heat_aquifer"] = y['heat_aquifer']

## Save CSV version of the dataset and simulation values

In [414]:
AVAILABLE_DATASET_PATHS = {OspitalettoDataset.OSPITALETTO: ospitaletto2019.processed_dataset_path,
                           NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path,
                           InsPireDataset.LONDON_UK: insPireDataset.processed_london_uk_dataset_path,
                           InsPireDataset.MADRID_SPA: insPireDataset.processed_madrid_spa_dataset_path,
                           InsPireDataset.ROME_IT: insPireDataset.processed_rome_it_dataset_path,
                           InsPireDataset.STUTTGART_GER: insPireDataset.processed_stuttgart_ger_dataset_path,}

Tamb_hourly.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])

In [415]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")

with open(simulation_values_path, "w") as f:
    json.dump(simulation_values, f, sort_keys=True, indent=2)